# インポート

In [5]:
import create_population
import numpy as np
import pandas as pd
import preprocessing
from pathlib import Path
import create_population
from feature_producing import FeatureCreator
from training import Trainer
from feature_producing_prediction import PredictionFeatureCreator
import predictioning
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
%autoreload

# 学習母集団の作成
期間を指定すると、学習母集団である（日付、race_id、horse_id）の組みを作る

In [66]:
population = create_population.create(
    from_ ="2023-01-01",
    to_ = "2023-12-31",
)

In [ ]:
# 各年の数を確認
population["date"].dt.year.value_counts().sort_index()

date
2018    48604
2019    47574
2020    48282
2021    47821
2022    47220
2023    47672
Name: count, dtype: int64

# データ加工

In [2]:
import preprocessing
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload

In [74]:
#レース結果テーブルの前処理
results_preprocessd = preprocessing.process_results()

In [13]:
# 重複チェック
results_preprocessd.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [76]:
# 欠損チェック
results_preprocessd.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
rank           0
umaban         0
wakuban        0
tansyo_odds    0
popularity     0
kinryou        0
sex            0
age            0
weight         0
weight_diff    0
dtype: int64

In [8]:
# 馬の過去成績テーブルの前処理
horse_results_preprocessd = preprocessing.process_horse_results()

In [9]:
horse_results_preprocessd

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_condition,race_class,n_horses
0,2021101429,2024-11-09,9.0,0.0,0.2,0.0,1,1400,0.0,4.0,14.0
1,2021101429,2024-10-06,1.0,1140.0,0.0,1.0,1,1400,0.0,3.0,13.0
2,2021101429,2024-09-01,2.0,625.2,0.4,0.0,1,1400,0.0,5.0,7.0
3,2021101429,2024-08-10,2.0,629.2,0.0,0.0,1,1400,0.0,5.0,14.0
4,2021101429,2024-04-06,13.0,0.0,0.9,1.0,1,1600,1.0,7.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
711582,2019100398,2022-01-27,2.0,52.0,0.4,1.0,0,1400,0.0,NaN,6.0
711583,2019100398,2022-01-04,2.0,48.0,0.0,0.0,0,1400,0.0,NaN,9.0
711584,2019100398,2021-12-16,1.0,70.0,0.0,0.0,0,1400,0.0,1.0,9.0
711585,2019100398,2021-10-26,2.0,14.0,0.1,1.0,0,1000,1.0,1.0,12.0


In [10]:
# 形式、欠損値を確認
horse_results_preprocessd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700274 entries, 0 to 711586
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   horse_id          700274 non-null  int64         
 1   date              700274 non-null  datetime64[ns]
 2   rank              700274 non-null  float64       
 3   prize             700274 non-null  float64       
 4   rank_diff         699553 non-null  float64       
 5   weather           699658 non-null  float64       
 6   race_type         700274 non-null  int64         
 7   course_len        700274 non-null  int64         
 8   ground_condition  700269 non-null  float64       
 9   race_class        337222 non-null  float64       
 10  n_horses          700266 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(3)
memory usage: 64.1 MB


In [83]:
# 欠損チェック
horse_results_preprocessd.isnull().sum()

horse_id                0
date                    0
rank                    0
prize                   0
rank_diff             260
weather               219
race_type               0
course_len              0
ground_condition        4
race_class          60543
n_horses                0
dtype: int64

In [16]:
%autoreload

In [17]:
# レース情報テーブルの加工
race_info_preprocess  = preprocessing.process_race_info()

In [12]:
race_info_preprocess

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,201801010101,2018-07-28,1,0.0,1800,0,0.0,1.0,1
1,201801010102,2018-07-28,0,0.0,1000,0,0.0,1.0,1
2,201801010103,2018-07-28,0,0.0,2400,0,0.0,1.0,1
3,201801010104,2018-07-28,0,0.0,1700,0,0.0,1.0,1
4,201801010105,2018-07-28,1,0.0,1500,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
20725,202310030808,2023-09-03,0,0.0,1000,0,0.0,2.0,10
20726,202310030809,2023-09-03,1,0.0,1200,0,0.0,3.0,10
20727,202310030810,2023-09-03,0,0.0,1700,0,0.0,4.0,10
20728,202310030811,2023-09-03,1,0.0,1200,0,0.0,6.0,10


In [18]:
# 欠損チェック
race_info_preprocess.isnull().sum()

race_id                0
date                   0
race_type              0
around               897
course_len           161
weather                0
ground_condition    4787
race_class          1886
place                  0
dtype: int64

# 特徴量作成

In [ ]:
from feature_producing import FeatureCreator
%load_ext autoreload

In [ ]:
%autoreload

In [88]:
fp = FeatureCreator()
features = fp.create_features()

In [89]:
# 重複チェック
features.duplicated(subset=["race_id", "horse_id"]).sum()

0

# 学習

In [ ]:
from training import Trainer
%load_ext autoreload

In [97]:
%autoreload

In [ ]:
trainer = Trainer()
%load_ext autoreload

In [ ]:
trainer = Trainer(config_filepath="config.yaml")
evaluation_df = trainer.run(
    valid_start_date="2022-10-01",
    test_start_date="2023-01-01",
    importance_filename="importance_new.csv",
    evaluation_filename="evaluation_new.csv"

[100]	training's binary_logloss: 0.153178	valid_1's binary_logloss: 0.206081


# 精度評価

In [1]:
from evaluationing import Evaluator
%load_ext autoreload

In [8]:
%autoreload

In [2]:
evaluator = Evaluator()

In [ ]:
evaluator = Evaluator(evaluation_filename="evaluation_5year.csv")


# 予測

## 事前準備
**当日出走馬が確定した時点**で実行できる

In [6]:
# 当日出走馬の過去成績テーブルの前処理
horse_results_preprocessed = preprocessing.process_horse_results(
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [ ]:
# 当日出走馬の血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds(
    input_filename="peds_prediction.csv",
    output_filename="peds_prediction.csv"
)

In [7]:
pfc = PredictionFeatureCreator(population_filename="population_5year.csv")
# 過去成績集計は事前に行うことができる
pfc.create_baselog()
pfc.agg_horse_n_races()
pfc.agg_horse_n_races_relative()
# pfc.agg_interval()

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\common\\data\\prediction_population\\population_5year.csv'

## 当日の予測処理
レース直前出走直前に実行する

In [ ]:
# 特徴量の更新
features = pfc.create_features(
    race_id="202405030101",  # 予測するレースidを指定
    skip_agg_horse=True  # 事前に集計した場合はスキップできる
)

fetching shubuta page html...


/Users/chiyoda/repositories/keibaAI-v3-yt/v3_1_0/src/feature_engineering_prediction.py:183: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...
running agg_sire()...
running cross_feature()...
merging all features...


In [ ]:
# 予測
prediction.predict(features)

,race_id,umaban,tansho_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
2,202405030101,3,121.9,8,0.005624,0.008065,0.685545,0.983065
0,202405030101,1,3.1,1,0.309638,0.306667,0.959879,0.950667
8,202405030101,9,7.5,5,0.112428,0.125828,0.843211,0.943709
3,202405030101,4,3.5,2,0.234721,0.244898,0.821522,0.857143
6,202405030101,7,6.4,4,0.125690,0.129032,0.804418,0.825806
9,202405030101,10,19.9,7,0.047463,0.037303,0.944514,0.742324
1,202405030101,2,5.1,3,0.160924,0.144981,0.820712,0.739405
4,202405030101,5,10.0,6,0.086425,0.068456,0.864247,0.684564
7,202405030101,8,163.3,10,0.004097,0.002841,0.668959,0.463920
5,202405030101,6,146.9,9,0.005314,0.002841,0.780582,0.417330


In [ ]:
# オッズ除外モデルでの予測
prediction.predict(
    features,
    model_filename="model_odds_removed.pkl",
    calibration_model_filename="calibration_model_odds_removed.pkl",
    config_filepath="config_odds_removed.yaml"
)

,race_id,umaban,tansho_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
5,202405030101,6,146.9,9,0.052190,0.038741,7.666716,5.691041
7,202405030101,8,163.3,10,0.041249,0.033053,6.735890,5.397591
2,202405030101,3,121.9,8,0.051555,0.035714,6.284495,4.353571
8,202405030101,9,7.5,5,0.121707,0.113924,0.912803,0.854430
4,202405030101,5,10.0,6,0.073797,0.081365,0.737972,0.813648
9,202405030101,10,19.9,7,0.041788,0.033053,0.831578,0.657759
1,202405030101,2,5.1,3,0.063697,0.066667,0.324854,0.340000
3,202405030101,4,3.5,2,0.089735,0.081365,0.314071,0.284777
0,202405030101,1,3.1,1,0.072417,0.081365,0.224494,0.252231
6,202405030101,7,6.4,4,0.048026,0.033053,0.307369,0.211541
